In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

第一步：加载数据

In [2]:
training_data  = datasets.FashionMNIST(
    root = "data",
    train=True,
    download =True,
    transform=ToTensor(),
)

test_data  = datasets.FashionMNIST(
    root = "data",
    train=False,
    download =True,
    transform=ToTensor(),
)

创建数据集

In [3]:
batch_size = 32
# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)
for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([32, 1, 28, 28])
Shape of y: torch.Size([32]) torch.int64


第二步：创建模型

In [4]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


第三步：训练模型<br>
创建损失函数和优化器<br>
这里的损失函数的数学公式：$L = -\log\left(\frac{\exp(x_{\text{class}})}{\sum_j \exp(x_j)}\right)$

In [5]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

定义一个模型训练函数

In [6]:
def train(dataloader,model,loss_fn,optimzer):
    size =len(dataloader.dataset)
    model.train()
    for batch,(x,y) in enumerate(dataloader):
        # X,y = X.to(device),y.to(device)
   
        #前向传播
        pred = model(X)
        loss =loss_fn(pred,y)
        
        #反向传播
        
        loss.backward() #计算当前的梯度
        optimizer.step() #更新参数
        optimizer.zero_grad() #清空之前的梯度

        if batch %100 == 0:
            loss,current = loss.item(),(batch+1)*len(X)
            print(f"loss:{loss:>7f} [{current:>5d}/{size:>5d}]")

第四步：评估模型的性能<br>
在测试集上评估模型的性能

In [7]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [8]:
epochs=5
for t in range(epochs):
    print(f"Epoxh{t+1}\n--------------")
    train(train_dataloader,model,loss_fn,optimizer)
    test(test_dataloader,model,loss_fn)
print("Done~!!!")

Epoxh1
--------------
loss:2.301723 [   32/60000]
loss:2.296839 [ 3232/60000]
loss:2.310529 [ 6432/60000]
loss:2.294062 [ 9632/60000]
loss:2.309073 [12832/60000]
loss:2.306443 [16032/60000]
loss:2.307686 [19232/60000]
loss:2.297992 [22432/60000]
loss:2.299429 [25632/60000]
loss:2.314402 [28832/60000]
loss:2.308901 [32032/60000]
loss:2.307748 [35232/60000]
loss:2.298169 [38432/60000]
loss:2.311879 [41632/60000]
loss:2.300685 [44832/60000]
loss:2.300408 [48032/60000]
loss:2.298439 [51232/60000]
loss:2.296381 [54432/60000]
loss:2.290705 [57632/60000]
Test Error: 
 Accuracy: 6.0%, Avg loss: 2.310426 

Epoxh2
--------------
loss:2.300835 [   32/60000]
loss:2.296547 [ 3232/60000]
loss:2.309724 [ 6432/60000]
loss:2.299176 [ 9632/60000]
loss:2.307570 [12832/60000]
loss:2.301347 [16032/60000]
loss:2.306018 [19232/60000]
loss:2.301201 [22432/60000]
loss:2.300141 [25632/60000]
loss:2.307016 [28832/60000]
loss:2.306643 [32032/60000]
loss:2.302117 [35232/60000]
loss:2.301242 [38432/60000]
loss:2.30

第五步：保存模型

In [9]:
torch.save(model.state_dict(), "model.pth")
print("Saved model")

Saved model


第六步：加载模型

In [10]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth",weights_only=True))

<All keys matched successfully>

第七步：应用模型进行预测

In [11]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "T-shirt/top", Actual: "Ankle boot"
